<h1><center>Laboratorio 5: Benchmark Bayesiano 🧮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Diego Irarrázaval

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Jonathan Espínola
- Nombre de alumno 2: Tomás Valencia


### **Link de repositorio de GitHub:** `https://github.com/Johnny952/labDataScience`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Objetivos principales del laboratorio](#Objetivos-principales-del-laboratorio)

# Temas a tratar

- Optimización de Código en Python.
- Utilización de librerías para medir el tiempo de ejecución de funciones.
- Métodos para optimizar el rendimiento de las funciones.

# Reglas:

- Fecha de entrega: 4/06/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

# Objetivos principales del laboratorio

- Obtener datos desde Reddit y visualizar cuales post son más probables que sean puntuados positivamente.
- Aplicar un atajo bayesiano para obtener la mean posterior de datos.
- Optimizar a través de librerías funciones.

#Importamos librerias utiles 😸

In [1]:
%%capture
!pip install praw
!pip install line_profiler
import sys
import praw
import time
import numpy as np
import pandas as pd
from numba import jit
import plotly.express as px
from functools import lru_cache
from IPython.core.display import Image

# 1. Recomendando Posts de Subreddit

<p align="center">
  <img src="https://styles.redditmedia.com/t5_2rer8/styles/bannerBackgroundImage_6o2td1zc54671.jpg?width=4000&format=pjpg&s=600bcf8560dff264f1cc7c0785ba5f6d529e0c28" width="10000">
</p>

Unos intrépidos alumnos del curso, quienes frecuentemente participan en subreddits y foros de reviews, se preguntan lo siguiente: ¿Podremos confiar que un post es bueno, si este tiene solamente 1 o 3 votos positivos?. los compañeros, creen que esto claramente no representa una opinión general, ya que estamos mucho menos seguros acerca de la verdadera proporción de votos a favor de los comentarios con pocos datos. ¿Pero cómo podemos obtener una representación más creíble para este problema?.

Lo señalado forma parte de un problema Bayesiano, donde a través del cálculo de la posterior se puede conocer que tan probable es que un post sea bueno. Para efectos de este laboratorio, no se exige un conocimiento previo para resolver este problema, simplemente se deberá aplicar las ecuaciones presentadas más adelante (De igual forma si quedan interesados sobre el tema se les invita a tomar el ramo [CC6104](https://github.com/dccuchile/CC6104)).

## 1.1 Obtención de Subrredits y Análisis

Para estudiar que tan probable es que un post sea positivo se comenzará cargando datos reales del subreddit chile desde reddit (si usted desea puede cambiar el subreddit a uno de su gusto). Para esto le proponemos la utilización de la función que aparece mas abajo, la que presenta un usuario ya creado por el equipo docente. Dese un tiempo para entender que hace cada parte de la función, visualizando que se obtiene de estas.

Revisada la función, utilice un **perfilador** para monitorear el tiempo y memoria que les toma a las subfunciones ser ejecutadas (por linea de código, vean las clases). Señale cuales son los procesos que mas tiempo consumen en la ejecución del Código, comentando si es posible mejorar el desempeño de la función.

**TO-DO:**
- [ ] Estudiar la función propuesta por el equipo docente.
- [ ] Estudiar los tiempos de ejecución del Código a través de un perfilador.

In [2]:
def praw_reddit(nombre_subreddit = 'chile', n_hot = 1000):
  reddit = praw.Reddit(client_id='-w2hyFINxZ8T3g',
                     client_secret='zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw',
                     password='ClasesMDS7202',
                     user_agent='Clases',
                     username='DocenciaDataScience', check_for_async=False)
  subreddit  = reddit.subreddit(nombre_subreddit)

  votes, post, url = {}, {}, {}
  top_submissions = list(subreddit.hot(limit = n_hot))
  for it, top_n in enumerate(range(50, len(top_submissions),50)):
    top_n_submissions = top_submissions[:top_n]
    upvotes, downvotes, url[it], post[it] = [], [], [], []

    for submission in top_n_submissions:
        try:
            ratio = submission.upvote_ratio
            ups = int(round((ratio*submission.score)/(2*ratio - 1)) if ratio != 0.5 else round(submission.score/2))
            upvotes.append(ups)
            downvotes.append(ups - submission.score)
            post[it].append(submission.title)
            url[it].append(submission.url)
        except Exception as e:
            continue
    votes[it] = np.array([upvotes, downvotes]).T
  return votes, post, url

In [ ]:
%load_ext line_profiler

In [ ]:
#### Código Aquí ####
%lprun -f praw_reddit praw_reddit()

**Respuesta:**

Timer unit: 1e-06 s

Total time: 17.2401 s
File: <ipython-input-2-d987b25ddeb0>
Function: praw_reddit at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def praw_reddit(nombre_subreddit = 'chile', n_hot = 1000):
     2         1          2.0      2.0      0.0    reddit = praw.Reddit(client_id='-w2hyFINxZ8T3g',
     3         1          2.0      2.0      0.0                       client_secret='zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw',
     4         1          1.0      1.0      0.0                       password='ClasesMDS7202',
     5         1          1.0      1.0      0.0                       user_agent='Clases',
     6         1      46226.0  46226.0      0.3                       username='DocenciaDataScience', check_for_async=False)
     7         1         31.0     31.0      0.0    subreddit  = reddit.subreddit(nombre_subreddit)
     8                                           
     9         1          1.0      1.0      0.0    votes, post, url = {}, {}, {}
    10         1   17102255.0 17102255.0     99.2    top_submissions = list(subreddit.hot(limit = n_hot))
    11        20         37.0      1.9      0.0    for it, top_n in enumerate(range(50, len(top_submissions),50)):
    12        19        182.0      9.6      0.0      top_n_submissions = top_submissions[:top_n]
    13        19        113.0      5.9      0.0      upvotes, downvotes, url[it], post[it] = [], [], [], []
    14                                           
    15      9519       8599.0      0.9      0.0      for submission in top_n_submissions:
    16      9500       8521.0      0.9      0.0          try:
    17      9500      12305.0      1.3      0.1              ratio = submission.upvote_ratio
    18      9500      16087.0      1.7      0.1              ups = int(round((ratio*submission.score)/(2*ratio - 1)) if ratio != 0.5 else round(submission.score/2))
    19      9500       9594.0      1.0      0.1              upvotes.append(ups)
    20      9500       9822.0      1.0      0.1              downvotes.append(ups - submission.score)
    21      9500      12523.0      1.3      0.1              post[it].append(submission.title)
    22      9500      12195.0      1.3      0.1              url[it].append(submission.url)
    23                                                   except Exception as e:
    24                                                       continue
    25        19       1582.0     83.3      0.0      votes[it] = np.array([upvotes, downvotes]).T
    26         1          1.0      1.0      0.0    return votes, post, url

`Los procesos que toman más tiempo es la línea 10 en que llama a la API, lo cual está limitado a los tiempos de respuesta del servidos, y el cálculo de metricas y guardado de posts/urls. El código puede ser optimizado, sabiendo que en la línea 12 se ve que se está pasando más de una vez por cada post, lo cual incrementa los tiempos de ejecución.`

## 1.2 Análisis de Tiempo con Cache

Sin duda, un factor clave en la mejora del tiempo de ejecución de una aplicación es el uso eficiente de la memoria. Por lo que es importante, que ustedes respondan las siguientes preguntas:
1.	¿Qué es la memoria cache y a que se refiere las siglas LRU?
`La memoria cache es una memoria de baja capacidad y rápido acceso, en la que se guardan datos más frecuentemente utilizados. LRU se refiere a kis algoritmos de cache que sirven para optimizar la gestión de información de la memoria caché, cuando esta se llena, elige que elementos eliminar y cuáles agregar.`
2.	¿Cuáles son los costos que tiene la aplicación de técnicas de Caching?
`Las técnicas de caching sirven para reducir los tiempos de ejecución, pero usando la memoria como costo.`
3. ¿Cuál es la consecuencía de ocupar caching en la función anterior?.
`Las consecuencias son de disminuir los tiempos de ejecución, aumentando el uso de memoria.`

Respondidas las preguntas, se le solicita que aplique alguna técnica de caching para mejorar el desempeño de la función `praw_reddit`. Para esto compare solo el tiempo de ejecución del algoritmo con y sin caching, señalando el tiempo total de ejecución y el tiempo promedio que le toma ejecutar cada loop a la función. Con lo anterior, ¿es posible visualizar mejoras en este caso?.


**TO-DO:**
- [ ] Responder preguntas.
- [ ] Mejorar el código con cache.


**Respuestas Teóricas:**

`Es posible visualizar una gran mejora en los tiempos de ejecución, ahora en algunos casos corre casi 10 veces más rápido que el anterior.`

In [3]:
#### Código para optimizar la función praw ####
def praw_reddit(nombre_subreddit = 'chile', n_hot = 1000):
  reddit = praw.Reddit(client_id='-w2hyFINxZ8T3g',
                     client_secret='zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw',
                     password='ClasesMDS7202',
                     user_agent='Clases',
                     username='DocenciaDataScience', check_for_async=False)
  subreddit  = reddit.subreddit(nombre_subreddit)
  cache_votes, cache_urls, cache_post = np.array([]).reshape(-1, 2), [], []

  votes, post, url = {}, {}, {}
  top_submissions = list(subreddit.hot(limit = n_hot))
  for it, top_n in enumerate(range(50, len(top_submissions),50)):
    top_n_submissions = top_submissions[top_n-50:top_n]
    upvotes, downvotes, url[it], post[it] = [], [], [], []
    post[it] += cache_post
    url[it] += cache_urls

    for submission in top_n_submissions:
        try:
            ratio = submission.upvote_ratio
            ups = int(round((ratio*submission.score)/(2*ratio - 1)) if ratio != 0.5 else round(submission.score/2))
            upvotes.append(ups)
            downvotes.append(ups - submission.score)
            post[it].append(submission.title)
            url[it].append(submission.url)
        except Exception as e:
            continue
    votes[it] = np.concatenate([cache_votes, np.array([upvotes, downvotes]).T], axis=0)
    cache_votes, cache_urls, cache_post = votes[it], url[it], post[it]

  return votes, post, url

In [ ]:
#### Código Aquí ####
%lprun -f praw_reddit praw_reddit()

Timer unit: 1e-06 s

Total time: 15.6968 s
File: <ipython-input-37-8238e0bedfcf>
Function: praw_reddit at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     2                                           def praw_reddit(nombre_subreddit = 'chile', n_hot = 1000):
     3         1          4.0      4.0      0.0    reddit = praw.Reddit(client_id='-w2hyFINxZ8T3g',
     4         1          2.0      2.0      0.0                       client_secret='zGPCI4s3g6Ic6AsRi7vIpP0NoxbFdw',
     5         1          1.0      1.0      0.0                       password='ClasesMDS7202',
     6         1          2.0      2.0      0.0                       user_agent='Clases',
     7         1        569.0    569.0      0.0                       username='DocenciaDataScience', check_for_async=False)
     8         1         27.0     27.0      0.0    subreddit  = reddit.subreddit(nombre_subreddit)
     9         1         17.0     17.0      0.0    cache_votes, cache_urls, cache_post = np.array([]).reshape(-1, 2), [], []
    10                                           
    11         1          1.0      1.0      0.0    votes, post, url = {}, {}, {}
    12         1   15685469.0 15685469.0     99.9    top_submissions = list(subreddit.hot(limit = n_hot))
    13        20         29.0      1.4      0.0    for it, top_n in enumerate(range(50, len(top_submissions),50)):
    14        19         41.0      2.2      0.0      top_n_submissions = top_submissions[top_n-50:top_n]
    15        19         39.0      2.1      0.0      upvotes, downvotes, url[it], post[it] = [], [], [], []
    16        19         73.0      3.8      0.0      post[it] += cache_post
    17        19         60.0      3.2      0.0      url[it] += cache_urls
    18                                           
    19       969        982.0      1.0      0.0      for submission in top_n_submissions:
    20       950        900.0      0.9      0.0          try:
    21       950       1479.0      1.6      0.0              ratio = submission.upvote_ratio
    22       950       1761.0      1.9      0.0              ups = int(round((ratio*submission.score)/(2*ratio - 1)) if ratio != 0.5 else round(submission.score/2))
    23       950       1057.0      1.1      0.0              upvotes.append(ups)
    24       950       1109.0      1.2      0.0              downvotes.append(ups - submission.score)
    25       950       1373.0      1.4      0.0              post[it].append(submission.title)
    26       950       1328.0      1.4      0.0              url[it].append(submission.url)
    27                                                   except Exception as e:
    28                                                       continue
    29        19        424.0     22.3      0.0      votes[it] = np.concatenate([cache_votes, np.array([upvotes, downvotes]).T], axis=0)
    30        19         24.0      1.3      0.0      cache_votes, cache_urls, cache_post = votes[it], url[it], post[it]
    31                                           
    32         1          1.0      1.0      0.0    return votes, post, url

## 1.3 Obtención de Mean Posterior y Standard Error

Una forma de obtener la mean posterior y el Standard Error de los datos de reddit es aplicando un atajo de inferencia Bayesiana. Este atajo se define de la siguiente forma:

Sea **u** los votos positivos y **d** los votos negativos del subreddit, tendremos que:

$$a = 1+u$$
$$b = 1+d$$

$$\sigma= 1.65\sqrt(\dfrac{ab}{(a + b)^2(a + b + 1)})$$

$$\mu = \dfrac{a}{a+b}$$

Donde $\mu$ es la mean posterior y $\sigma$ el standard error.

Con lo anterior, genere dos funciones que tengan como salida $\mu$ y $\sigma$ de acuerdo a las ecuaciones señaladas. La primera función, deberá ser construida sin el uso de numpy, aplicando for y aplicando comandos nativos de Python. Por otro lado, deberá generar una segunda función con el uso exclusivo de numpy. **OJO** que las funciones deben tener como entrada solo un elemento del diccionario votes (por ejemplo `votes[1]`), por lo que estas no deben tener como entrada el conjunto completo de datos.

In [10]:
def intervalos(votos_positivos, votos_negativos):
  mhus = []
  sigmas = []
  for i in range(votos_positivos.shape[0]):
    a = votos_positivos[i] + 1
    b = votos_negativos[i] + 1
    mhu = a / (a + b)
    den = (a + b)**2 * (a + b + 1)
    num = (a * b)
    sigma = 1.65 * (num/den)**0.5
    mhus.append(mhu)
    sigmas.append(sigma)
  return mhus, sigmas

def intervalos_numpy(votos_positivos, votos_negativos):
  a = votos_positivos + 1
  b = votos_negativos + 1
  mhu = a / (a + b)
  den = np.power(a + b, 2) * (a + b + 1)
  num = (a * b)
  sigma = 1.65*np.sqrt(num/den)
  return mhu, sigma


## 1.4 Comparación de rendimiento

Creadas las funciones de la sección 1.3, de forma iterativa compare los diferentes batches de datos que contiene el diccionario `votes`. Para esto genere un gráfico utilizando plotly, donde se pueda observar las diferencias de tiempo que toma ejecutar las diferentes cantidades de datos. ¿es posible observar una diferencia? ¿a qué se deberá esto?.

Aplique el compilador **Numba** sobre las funciones 1.3 y compare el tiempo de ejecución con los obtenidos sin el compilador. Grafique estos tiempos y observe comente los desempeños.

In [13]:
###### Código Aquí ######

@jit(nopython=True)
def intervalos_JIT(votos_positivos, votos_negativos):
  mhus = []
  sigmas = []
  for i in range(votos_positivos.shape[0]):
    a = votos_positivos[i] + 1
    b = votos_negativos[i] + 1
    mhu = a / (a + b)
    den = (a + b)**2 * (a + b + 1)
    num = (a * b)
    sigma = 1.65 * (num/den)**0.5
    mhus.append(mhu)
    sigmas.append(sigma)
  return mhus, sigmas

@jit(nopython=True)
def intervalos_numpy_JIT(votos_positivos, votos_negativos):
  a = votos_positivos + 1
  b = votos_negativos + 1
  mhu = a / (a + b)
  den = np.power(a + b, 2) * (a + b + 1)
  num = (a * b)
  sigma = 1.65*np.sqrt(num/den)
  return mhu, sigma


In [14]:
votes, post, url = praw_reddit()

In [19]:

db = []

for key_votos, votos in votes.items():
  tic = time.process_time()
  _, _ = intervalos(votos[:, 0], votos[:, 1])
  db.append([
             'Python puro', 
             time.process_time() - tic,
             key_votos
             ])

  tic = time.process_time()
  _, _ = intervalos_numpy(votos[:, 0], votos[:, 1])
  db.append([
             'Numpy', 
             time.process_time() - tic,
             key_votos
             ])

  tic = time.process_time()
  _, _ = intervalos_JIT(votos[:, 0], votos[:, 1])
  db.append([
             'Numba', 
             time.process_time() - tic,
             key_votos
             ])

  tic = time.process_time()
  _, _ = intervalos_numpy_JIT(votos[:, 0], votos[:, 1])
  db.append([
             'Numba-Numpy', 
             time.process_time() - tic,
             key_votos
             ])

In [20]:
df = pd.DataFrame(data=db, columns=["tipo", "valor", "llave"])
fig = px.line(df, x="llave", y="valor", title='Tiempos de ejecución', color="tipo")
fig.show()

`En el gráfico se puede ver que hay una clara diferencia entre las curvas o tiempos de ejecución para cada uno de los métodos, donde el más ineficiente es python sin librerías, Numpy, Numba y Numpy-Numba tienen tiempos similares, pero este último tiene mayor eficiencia. Esto se debe a que python utiliza sólo 1 núcleo de CPU, mientras que numpy usa más de uno de forma eficiente y numba utiliza la GPU, la cual tiene más núcleos que la CPU, y, por tanto, mayor paralelismo.`

## 1.5 Plot de Resultados Bayesianos

Llego la hora de visualizar los resultados obtenidos, para esto solo ejecute las siguientes celdas y observe lo que sucede :3. ¿Qué logra observar de los resultados?, ¿La solución resulta trivial?

En esta sección esperamos que solo comenten con lo que logran visualizar de los dataframes.

In [11]:
votos = votes[len(votes)-1]
posteo = post[len(post)-1]
print("lower bounds aproximados:")
posterior_mean, std_err = intervalos_numpy(votos[:, 0], votos[:, 1])
lb = posterior_mean - std_err
print("\nTop 20 post ordenador por el limite inferior:\n")
order = np.argsort(-lb)
vote_post = {'Votos (+)':votos[order[:20], 0], 'Votos (-)':votos[order[:20], 1] ,'Post':np.array(posteo)[order[:20]], 'url': np.array(url[len(url)-1])[order[:20]]}
df = pd.DataFrame(data=vote_post)
ordered_post = df.Post
df

lower bounds aproximados:

Top 20 post ordenador por el limite inferior:



,Votos (+),Votos (-),Post,url
0,934.0,0.0,Gato andino extremadmente raro mirando una cám...,https://v.redd.it/k2r2bhlh0h971
1,720.0,0.0,"Debate presidencial 2005, pregunta clave de Ma...",https://v.redd.it/xj4g8rc5sw671
2,522.0,0.0,¿Qué hacer con Talca? Bueno...,https://i.redd.it/1tk589eq89771.png
3,453.0,0.0,"El bombardero de Lo Prado: ""Se lo mando seco e...",https://v.redd.it/l6c86bhd2w571
4,446.0,0.0,"Aysén hoy 2, no se subieron todas juntas (Bonu...",https://www.reddit.com/gallery/o6f7lr
5,406.0,0.0,"Juan Carlos Bodoque, This Charming Man (Video ...",https://v.redd.it/wf0yqt92tv771
6,354.0,0.0,Fotografía de un vendedor ambulante en la Plaz...,https://www.reddit.com/gallery/oc0qzt
7,348.0,0.0,Avión Pingüino Rey de Aerovías DAP que vuela e...,https://i.redd.it/f82si2h743771.jpg
8,270.0,0.0,Cuando el chico Paris y Siches eran compañeros...,https://v.redd.it/nkfiwmwgma571
9,250.0,0.0,Con 99% de aprobación: Bomberos es la instituc...,https://www.24horas.cl/nacional/bomberos-aprob...


In [12]:
r_order = order[::-1][-20:]
data_dic = {'mean':posterior_mean[r_order], 'std_err':std_err[r_order], 'post':ordered_post[::-1]}
df = pd.DataFrame(data=data_dic)
fig = px.scatter(df, x="mean", y="post",
                 error_x="std_err")
fig.show()

**Respuesta:**

Al mirar los datos se puede ver que están ordenados por el límite enferior. 

La solución no resulta trivial, ya que no es claro cuál es el mejor post, el que tiene 1529 upvotes y 15 downvotes o el que tiene solo 214 upvotes.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab, solo para tareas**. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>